# 2D Filter demo

## Requirements

Custom video overlay for Pynq-Z1 v2.0 image
HDMI In connecte to a 1280x720 (720p) source
HDMI Out of the board connected to a monitor capable of displaying 720p

This demo allows the weights of a 3 channel 5x5 convolutional filter to be modified 


## Slideshow

* If the resolution is incorrectly set the first time the board is connected, set the resolution, and rerun all the cells in this notebook
 
This demo is intended to be run as a slideshow. 

Go to View > Cell Toolbar > Slideshow to see the *Slideshow options* for each cell. From here you can select which slides will be included or excluded from the presentation. 

Code cells can be executed from the slideshow view by clicking the code cell, and pressing CTRL + ENTER


## Instructions to run the demo:

* Before entering the slideshow view, click Cell > Run All to execute all the code in the notebook. 

In slideshow mode, use the cursor keys (left right) to navigate through the presentation

* Press Alt + r to enter the slideshow and hide this view. (Exit slideshow mode with the same keys Alt + r)

In [1]:
import pynq
ol = pynq.Overlay('./filter.bit')

from pynq.lib.video import *
rgba = PixelFormat(32, COLOR_IN_RGB, COLOR_OUT_RGB)
mode = VideoMode(1280, 720, 32)

ol.video.hdmi_in.configure(rgba)

ol.video.hdmi_in.start()

ol.video.hdmi_out.configure(mode, rgba)
ol.video.hdmi_out.start()

Check the input resolution. 

In [2]:
ol.video.hdmi_in.mode # Check input mode; Should be 1280x 720

VideoMode: width=1280 height=720 bpp=32

This should turn on the monitor but nothing will be displayed yet. 

In [3]:
filter2d = ol.video.filter2d_0

coeffs = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)

The arguments to `np.frombuffer` are the buffer object, the type we want, the number of elements and the offset in bytes. 96 is chosen because that's the number of 16-bit integers that can fit into the range 0x40 to 0xFF and 64 is the offset of the coeffs1 array.

Now we have the master array we can get the slices for each particular set of coefficients.

In [4]:
c1 = coeffs[0:25].reshape((5,5))
c2 = coeffs[32:57].reshape((5,5))
c3 = coeffs[64:89].reshape((5,5))

Note that while we have specified the types here as a 16-bit integer the type used by the hardware is a 16-bit fixed-point number so we need multiply the value of the coefficients by 256 when writing them to the hardware. Finally we can make our filter pass through the signal intact and (hopefully) see something on the screen.

In [6]:
def update_filter(sender):
    for i in range(5):
        for j in range(5):
            c1[(i,j)] = my_coefficients[i*5+j].value
            c2[(i,j)] = my_coefficients[25 + i*5+j].value
            c3[(i,j)] = my_coefficients[50 + i*5+j].value

Connect HDMI in to HDMI out. Make sure something is being displayed from the HDMI source to the board.

In [7]:
ol.video.hdmi_in.tie(ol.video.hdmi_out)

Build integer input box widgets to control the filter coefficinets. 

In [8]:
import ipywidgets as widgets
from ipywidgets import interact, Layout
from IPython.display import display
from ipywidgets import HBox, VBox

my_coefficients = []   
for i in range(3):
    for j in range(25):
        my_coefficients.append(widgets.IntText(value=0, disabled=False, layout=Layout(width='50px')))
        my_coefficients[i*25+j].observe(update_filter)

Add labels using HTML components

In [9]:
red = widgets.HTML(
    value="Red Channel",
    placeholder='Red html'
)

green = widgets.HTML(
    value="Green Channel",
    placeholder='Green html'
)

blue = widgets.HTML(
    value="Blue Channel",
    placeholder='Blue html'
)

labels = [red, green, blue]

# Create some example filters

In [ ]:
# Some options for filters: https://en.wikipedia.org/wiki/Kernel_%28image_processing%29
    

default_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
default_mask = [[0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0,256, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0]]

blank_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
blank_mask = [[0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0]]

sobel_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
sobel_mask = [[0,0,  1,0,0],
              [0,1,  2,1,0],
              [1,2,-16,2,1],
              [0,1,  2,1,0],
              [0,0,  1,0,0]]
sobel_mask = 256*np.array(sobel_mask)

enhance_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
enhance_mask = [[0, 0,  5, 0, 0],
                [0, 5,  9, 5, 0],
                [5, 9,256, 9, 5],
                [0, 5,  9, 5, 0],
                [0, 0,  5, 0, 0]]

sharpen_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
sharpen_mask = [[0, 0,  0, 0, 0],
                [0, 0, -1, 0, 0],
                [0,-1,  5,-1, 0],
                [0, 0, -1, 0, 0],
                [0, 0,  0, 0, 0]]
sharpen_mask = 256*np.array(sharpen_mask)

gaussian_blur_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
gaussian_blur_mask = [[1, 4,  6, 4, 1],
                      [4,16, 24,16, 4],
                      [6,24, 36,24, 6],
                      [4,16, 24,16, 4],
                      [1, 4,  6, 4, 1]]

# Create function to update coefficients

The update function will be called when values are updated in the widget boxes

In [10]:
def update_coeffs(sender):
    if presets.value is 'Default':
        c1[:] = default_mask[:]
        c2[:] = default_mask[:]
        c3[:] = default_mask[:]
    elif presets.value is 'Sobel':
        c1[:] = sobel_mask[:]
        c2[:] = sobel_mask[:]
        c3[:] = sobel_mask[:]
    elif presets.value is 'Warm':
        c1[:] = enhance_mask[:]
        c2[:] = default_mask[:]
        c3[:] = default_mask[:]
    elif presets.value is 'Cool':
        c1[:] = default_mask[:]
        c2[:] = default_mask[:]
        c3[:] = enhance_mask[:]
    elif presets.value is 'Sharpen':
        c1[:] = sharpen_mask[:]
        c2[:] = sharpen_mask[:]
        c3[:] = sharpen_mask[:]
    elif presets.value is 'Blur':
        c1[:] = gaussian_blur_mask[:]
        c2[:] = gaussian_blur_mask[:]
        c3[:] = gaussian_blur_mask[:]
    elif presets.value is 'Red_Edge':
        c1[:] = sobel_mask[:]
        c2[:] = blank_mask[:]
        c3[:] = blank_mask[:]
    elif presets.value is 'Blue_Edge':
        c1[:] = blank_mask[:]
        c2[:] = blank_mask[:]
        c3[:] = sobel_mask[:]
    elif presets.value is 'Green_Edge':
        c1[:] = blank_mask[:]
        c2[:] = sobel_mask[:]
        c3[:] = blank_mask[:]
    
presets = widgets.Select(
    options=['Default', 'Sobel', 'Sharpen', 'Blur', 'Warm', 'Cool', 'Red_Edge', 'Blue_Edge', 'Green_Edge'],
    value='Default',
    # rows=10,
    description='Presets:',
    disabled=False,
    layout=Layout(width='250px')
)
presets.observe(update_coeffs)
update_coeffs(_)

# Create widget layout

In [11]:
import traitlets

my_hboxes = [None] *7
my_vboxes = [None] *3
for i in range(3):
    my_hboxes[0] = labels[i]
    for j in range(5):
        my_hboxes[j+1] = widgets.HBox(my_coefficients[i*25 + j*5: i*25 + j*5 +5])
    my_hboxes[6] = presets
    my_vboxes[i] = widgets.VBox(my_hboxes, layout=Layout(border='solid 2px', width= '280px'))

my_boxes = widgets.HBox(my_vboxes)

# Video filter

The boxes below are weights for the convolutional filter. Chose from presets, or set your own values by typing numbers into the box. 

* Hint, the value in the centre box will increase or reduce the strength of the colour
* Moving the default value (256) around the box will shift the color up/down/left/right slightly
* You can also select some presets fromt he drop down box

In [12]:
display(my_boxes)

If resetting the notebook, the functions below should be uncommented and executed to free memory and stop the HDMI controllers gracefully. If these fucntions are not run, and rerunning the notebook gives a memory error, or if the PL stops responding, restart the board. 

In [14]:
#ol.video.hdmi_out.stop()
#ol.video.hdmi_in.stop()